In [1]:
chars = ""
with open('/kaggle/input/dataset/wizard_oz.txt', 'r', encoding='utf-8') as f:
    text=f.read()
    chars = sorted(list(set(text)))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [2]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

cuda


In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[68, 57, 78,  ...,  1, 78, 68],
        [73,  0, 76,  ..., 61, 58, 72],
        [ 1, 59, 71,  ..., 61, 58,  1],
        ...,
        [54, 67,  1,  ..., 68, 69,  1],
        [68, 67, 60,  ...,  1, 73, 61],
        [67, 57,  1,  ..., 73, 54, 71]], device='cuda:0')
targets:
tensor([[57, 78,  1,  ..., 78, 68, 74],
        [ 0, 76, 68,  ..., 58, 72, 58],
        [59, 71, 68,  ..., 58,  1, 76],
        ...,
        [67,  1, 76,  ..., 69,  1, 73],
        [67, 60, 58,  ..., 73, 61, 58],
        [57,  1, 73,  ..., 54, 71, 73]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



fnGyvh RfkAP'h"C6gibEsxnNI2;iAoIx9eg
7*TGu[i_b﻿J5 3Z*q9U7s_DWLdKT6-oTGHivB1s1OPA8"*0mH!Ed:eEWngAvQRr8ek2fL﻿0vr9DEBT79xJdsL(zv"sx_*tuL!kM9BAVr"P'7XDZmyXrWj&cqD?sP's_0'q12K]wYLgMF0Yc89QUZ'*ZQG8)2s_0mK9MHC5*Tp]9I]CwhbhJmxgk3FM,wsgk'w-cTxE2XoGo2,]1h;ISiA
jAqRR"5.RH]BLi6-]MYGsMY1U7(*rMMpk
zoATh DmK9iTwdK9]VEt3R-"NBV"M;)F.J47v,,c_I:9﻿o.1s0XLgv,HSYo)rrMM﻿mxiU;iaPaIFXiOp]0PC'[11ARuuU]iTTwl'nCFS*.l;r8VMU5 'lm;j.:Bf,K[gEmcaSgiF"UL
jQ.P9J_m.O]:dzo*,WN1p8emV6j?Y.vzmxZ6-]osAGEx&SB﻿p(gv70MZ0g(*GYgumxwX467(S)j


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.928, val loss: 4.933
step: 100, train loss: 4.879, val loss: 4.886
step: 200, train loss: 4.832, val loss: 4.840
step: 300, train loss: 4.787, val loss: 4.793
step: 400, train loss: 4.741, val loss: 4.748
step: 500, train loss: 4.697, val loss: 4.705
step: 600, train loss: 4.654, val loss: 4.661
step: 700, train loss: 4.611, val loss: 4.618
step: 800, train loss: 4.566, val loss: 4.573
step: 900, train loss: 4.522, val loss: 4.534
step: 1000, train loss: 4.482, val loss: 4.493
step: 1100, train loss: 4.441, val loss: 4.450
step: 1200, train loss: 4.399, val loss: 4.412
step: 1300, train loss: 4.361, val loss: 4.370
step: 1400, train loss: 4.319, val loss: 4.330
step: 1500, train loss: 4.281, val loss: 4.292
step: 1600, train loss: 4.242, val loss: 4.254
step: 1700, train loss: 4.203, val loss: 4.215
step: 1800, train loss: 4.166, val loss: 4.179
step: 1900, train loss: 4.131, val loss: 4.141
step: 2000, train loss: 4.093, val loss: 4.105
step: 2100, train loss: 4

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 D;nqcuyWh9&2mG!*r6GuP*roC6,-m'uP_M!:51BWszo)IeQo(1ARBW'.IY(992,VnC5uimy;pfNolL, Q[gA)tonid &T3qFVWQdiMHuTh4X(-ty-Ev"L[W6jqvavZ,"YL: D3suM*0 ﻿G.,2VEGu
aEM*!"Tf1owV8EeM .StPd9Bo,"SGwr6-rk6-?E s ﻿d 2JU8GNSBXX up_ sG!!lxcup!)Q9B-9By_t8-!IS;SOD PgE.X-yYIw)RF55[oE.l:eFX9D03G
TG"cBTGa
u D'QsL9lLa*eDXoZ*7byvhQscarORfekPCarb.Y.lE-K?H!haluvE'u1T6O-D EeC3ximtyqD9(WO7Ho&BftscakvBaRRMC224OzaK]smY?
92;Ptve&PdMt Jo?, wnJ8L,Tf﻿[﻿o[qkz:]heW6kGoa)H!-tP)sXr
6y:H1pJU[AULCKv_w:BHkAV"M!Grqcq)Gy :]VtuuYo &onds"4X-h-c


**GPT - STARTER**

In [10]:
#<======Head========>
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v
        return out

    
#<======MultiHeadAttention========>
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

    
#<======FeedForward========>    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
    def forward(self, x):
        return self.net(x)

    
#<======Block========>     
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

    
#<======GPTLanguageModel========>     
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape
        
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

   

model = GPTLanguageModel(vocab_size)
m = model.to(device)
    

In [11]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.403, val loss: 4.411
step: 100, train loss: 2.302, val loss: 2.367
step: 200, train loss: 1.856, val loss: 1.987
step: 300, train loss: 1.611, val loss: 1.769
step: 400, train loss: 1.479, val loss: 1.678
step: 500, train loss: 1.367, val loss: 1.611
step: 600, train loss: 1.287, val loss: 1.547
step: 700, train loss: 1.220, val loss: 1.513
step: 800, train loss: 1.166, val loss: 1.513
step: 900, train loss: 1.113, val loss: 1.505
step: 1000, train loss: 1.067, val loss: 1.494
step: 1100, train loss: 1.016, val loss: 1.513
step: 1200, train loss: 0.975, val loss: 1.515
step: 1300, train loss: 0.925, val loss: 1.524
step: 1400, train loss: 0.883, val loss: 1.541
step: 1500, train loss: 0.836, val loss: 1.565
step: 1600, train loss: 0.791, val loss: 1.586
step: 1700, train loss: 0.752, val loss: 1.612
step: 1800, train loss: 0.708, val loss: 1.621
step: 1900, train loss: 0.678, val loss: 1.667
step: 2000, train loss: 0.630, val loss: 1.710
step: 2100, train loss: 0